In [13]:
import ete3
import os
import json

In [14]:
mapping_file = '../data/conifer_mapping.json'
name_mapping = json.load(open(mapping_file))
genera = ['Amentotaxus','Austrotaxus','Cephalotaxus','Taxus','Torreya','Pseudotaxus']

pruned_names = []
for n in genera:
    for k,v in name_mapping.iteritems():
        if v.startswith(n):
            pruned_names.append(str(k))
print pruned_names, 

['IAJW', 'BTTS', 'NVGZ', 'WWSS', 'ZYAX', 'EFMS', 'HQOM', 'YLPM']


In [15]:
def get_pruned_names_abbr(t, pruned_names):
    r = []
    mapping = {}
    for leaf in t:
        for n in pruned_names:
            if leaf.name.startswith(n):
                r.append(leaf.name)
                mapping[leaf.name] = name_mapping[n]
    return r, mapping

In [25]:
directory = '../data/conifers/'

trees = {}
cnt = 0
for dirpath, dirnames, filenames in os.walk(directory):
    for filename in filenames:
        if filename.endswith('nwk'):
            with open(os.path.join(dirpath, filename)) as f:
                trees[filename] = ete3.Tree(f.readline().strip(), format=5)
                p, mapping = get_pruned_names_abbr(trees[filename], pruned_names)
                try:
                    trees[filename].prune(p)
                    #for leaf in trees[filename]:
                    #    leaf.name = mapping[leaf.name]
                    cnt += 1
                except:
                    print filename
                    del trees[filename]     
                       

all-conifers_5341.filtered.fna.cleaned.muscle_aln.trimmed.fasttree_nwk


In [26]:
print cnt
output_dir = '../data/pruned_conifers_sample/'
print trees['all-conifers_4525.filtered.fna.cleaned.muscle_aln.trimmed.fasttree_nwk'].write(format=5)
print len(trees.keys())

3159
((NVGZ_2008606:0,NVGZ_2008607:0):0.05102,((EFMS_2009672:0.02161,IAJW_2020179:0.01581):0.01974,(ZYAX_2073852:0.02392,(BTTS_2076622:0.00054,YLPM_2007099:0.00055):0.0157):0.02363):0.00425);
3159


In [31]:
def has_duplicate(tree):
    name_hash = {}
    for leaf in tree:
        x = leaf.name[:4]
        if x in name_hash:
            return True
        name_hash[x] = True


In [34]:
cnt = 0
for k,v in trees.iteritems():
    if not has_duplicate(v):
        cnt += 1
        for leaf in v:
            leaf.name = name_mapping[leaf.name[:4]]
        v.write(outfile=os.path.join('../data/pruned_conifer_single_gene/', 
                                     k.replace('all-conifers', 'pruned-conifers')), format=5)

print cnt

1836


In [27]:
i = 0
for k,v in trees.iteritems():
    v.write(outfile=os.path.join(output_dir, k.replace('all-conifers', 'pruned-conifers')), format=5)
    i += 1
    if i > 999:
        break